In [1]:
from linares_plot import *

In [2]:
data = pd.read_csv('C:\\Users\\David\\Dropbox\\IDIBAPS\\my code\\Project 1 (master thesis)\\idibaps_psychophysics')

In [4]:
df_controls = data.loc[data['type']==1]
df_controls

,Unnamed: 0,r_T,r_NT,delay,type,A_T,A_NT,A_err,r_err,A_R,r_R,subject,session,Int_A_err,Int_r_err
3,3,13.68,13.68,3.0,1.0,227.0,215.0,-3.86,2.12,230.86,11.56,1,1,-3.86,NaN
10,10,13.68,13.68,3.0,1.0,41.0,29.0,-4.33,2.97,45.33,10.71,1,1,-4.33,NaN
23,23,13.68,13.68,3.0,1.0,214.0,194.0,-2.10,1.89,216.10,11.79,1,1,-2.10,NaN
24,24,7.72,7.72,0.0,1.0,232.0,212.0,2.09,0.94,229.91,6.78,1,1,2.09,NaN
36,36,7.72,7.72,0.0,1.0,43.0,31.0,-4.59,1.29,47.59,6.43,1,1,-4.59,NaN
38,38,13.68,13.68,0.0,1.0,213.0,201.0,-9.85,0.97,222.85,12.71,1,1,-9.85,NaN
43,43,7.72,7.72,0.0,1.0,149.0,137.0,-6.52,0.51,155.52,7.21,1,1,-6.52,NaN
44,44,7.72,7.72,3.0,1.0,50.0,34.0,-2.61,1.93,52.61,5.79,1,1,-2.61,NaN
51,51,7.72,7.72,0.0,1.0,38.0,22.0,-7.38,1.51,45.38,6.21,1,1,-7.38,NaN
54,54,7.72,7.72,0.0,1.0,145.0,125.0,-9.46,0.38,154.46,7.34,1,1,-9.46,NaN


In [ ]:
dfr=data.loc[(data['type']==3) | (data['type']==4)]

in_out=[]
### in --> the target is closer to fixation
### out --> the target is far from fixation

for i in dfr.index:
    if dfr.loc[i, 'r_T']<dfr.loc[i, 'r_NT']:
        in_out.append('in')
    else:
        in_out.append('out')


dfr['in_out'] = in_out


print(len(dfr))

def mod(target,report,radius_t, radius_r):
    #units given in degrees and cm
    y_t=np.array(radius_t*np.sin(np.radians(target)),dtype=np.float)
    x_t=np.array(radius_t*np.cos(np.radians(target)),dtype=np.float)
    y_r=np.array(radius_r*np.sin(np.radians(report)),dtype=np.float)
    x_r=np.array(radius_r*np.cos(np.radians(report)),dtype=np.float)
    m = np.sqrt((x_t - x_r)**2 + (y_t - y_r)**2)
    return m


modules = [round(mod(dfr.loc[i,'A_T'], dfr.loc[i,'A_R'], dfr.loc[i,'r_T'], dfr.loc[i,'r_R']),3) for i in dfr.index]
dfr['modules']=modules


from pandas import concat
frames=[]
for group_label in dfr.subject.unique():
    sub_group=dfr.groupby('subject').get_group(group_label)
    #CI
    #interval = sub_group.quantile([0.025, 0.975]).unstack(level=1)['A_err']
    #removed_out = sub_group.loc[(sub_group['A_err']>interval.iloc[0]) & (sub_group['A_err']<interval.iloc[1])] 
    #
    #Quartiles
    limit=sub_group['modules'].mean() + 2*sub_group['modules'].std()
    #print(round(limit,2))
    removed_out =sub_group.loc[sub_group['modules']<limit  ]
    
    
    frames.append(removed_out)
    
    #Print the result
    before_outliers=len(sub_group)
    after_outliers=len(removed_out)
    print( 'Outliers in ' + str(group_label) + ' = ' +str(before_outliers-after_outliers) + ', limit=' +str(round(limit,2))  + 'cm')
    


dfr = concat(frames)
print(len(dfr))

### Distribution A_Err in cw-ccw
sns.distplot(dfr.loc[ dfr['r_T']==7.72, 'r_R'], color='b', label='7.72')
sns.distplot(dfr.loc[ dfr['r_T']==10.7, 'r_R'], color='g', label='10.7')
sns.distplot(dfr.loc[ dfr['r_T']==13.68, 'r_R'], color='r', label='13.68')
plt.title('Distribution radial response')
plt.legend(frameon=False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().get_xaxis().tick_bottom()
plt.gca().get_yaxis().tick_left()
plt.xlabel('radial response')

plt.plot([7.72, 7.72], [0, 0.4], 'k--')
plt.plot([10.7, 10.7], [0, 0.4], 'k--')
plt.plot([13.68, 13.68], [0, 0.4], 'k--')
plt.show()

sns.pointplot(x='r_T', y='r_err', hue='delay', x_estimator=np.mean, legend=False, data=dfr)
plt.legend(frameon=False)
plt.title('Drift to fixation')
plt.xlabel('Radius target')
plt.ylabel('Attraction fixation (cm)')
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().get_xaxis().tick_bottom()
plt.gca().get_yaxis().tick_left()
plt.show()



dfr['drift_norm'] = dfr['r_err']  / dfr['r_T']
sns.pointplot(x='r_T', y='drift_norm', hue='delay', x_estimator=np.mean, legend=False, data=dfr)
plt.legend(frameon=False)
plt.title('Drift to fixation normalized')
plt.xlabel('Radius target')
plt.ylabel('Attraction fixation (cm)')
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().get_xaxis().tick_bottom()
plt.gca().get_yaxis().tick_left()
plt.show()